In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
import numpy as np
import pandas as pd

import os


import math


from scipy.optimize import curve_fit




from JJformulas import *

In [2]:
import sys 
sys.path.append('..\measDC\_jjtools')

# from Exps import *

from JJ_data_processing import read_opj_data, pbi, bpbi, show_df, update_df
from JJ_data_processing import xy_by_id,extract_Isw_R0_by_id


from JJformulas import *
from meas_util import *

from JJmeasurement import *

D:\anaconda\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'
D:\anaconda\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Samples

In [3]:
jj70N5 = JJmeas(sample = 'D070B1N5v1')

jj70N5.db_connect()

In [4]:
jj70N1 = JJmeas(sample = 'D070B1N1v1')

jj70N1.db_connect()

In [5]:
jj70N0 = JJmeas(sample = 'D070B1N0v1')

jj70N0.db_connect()

In [6]:

jj70N7 = JJmeas(sample = 'D070B1N7')

jj70N7.db_connect()

In [7]:
jj85 = JJmeas(sample = 'D085N5')
jj85.db_connect()



In [8]:
jj79 = JJmeas(sample = 'D079N6')
jj79.db_connect()

In [9]:
jj59 = JJmeas(sample = 'D059BBN1')
jj59.db_connect()

In [30]:
jj57 = JJmeas(sample = 'D057B1N8v1')
jj57.db_connect()

In [10]:
jj70N5.pbi(1, interactive = True)

FigureCanvasNbAgg()

# Some coding

In [69]:
from scipy import interpolate

fig, ax = plt.subplots(figsize = (4,5))

def gaussian(x, mu, sig):
    return 1./(np.sqrt(2.*np.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2.)/2)

Rsh = 377

kT = 0.050
Ec = 1
EJ = 0.4
Vc_noise = np.sqrt(2*Ec*kT)/1e4


Npts = 501
V_ = np.linspace(-1e-3, 1e-3, Npts)/1e1


def IZ_conv(V_, EJ, Vc_noise, kT, Rsh):
    
    sigma = Vc_noise#np.sqrt(Ec*kT)/1e4
    Is = I_IZ( V_, EJ = EJ, R = Rsh, T = kT)

    Vs = V_ - Is*Rsh

    f = interpolate.interp1d(Vs, Is)




    Vs_evengrid = np.linspace(np.min(Vs), np.max(Vs), len(Vs))
    
    dVs =  (np.max(Vs) - np.min(Vs)) / (len(Vs)-1)
    
    

    Is_evengrid = f(Vs_evengrid)

    Is = Is_evengrid
    Vs = Vs_evengrid

    V_c = gaussian(Vs, 0, sigma)*dVs

    Is_conv = np.convolve(Is, V_c, 'same')
#     print(len(Is))
    
    return Vs, Is, Is_conv

Vs, Is, Is_conv = IZ_conv(V_, EJ, Vc_noise, kT, Rsh)


ax.plot(Vs/1e-6, Is/1e-9, 'o', label = 'cIZ')

ax.plot(Vs/1e-6 ,Is_conv/1e-9, label = 'cIZ + Vnoise')



ax.set_xlabel('V, uV')
ax.set_ylabel('I, nA')

ax.set_title('EJ = {:g} K, T = {:g} K, Rb = {:g} Ohm'.format(EJ, kT, Rsh))

# ax.set_xlim(0, 300)
# ax.set_ylim(0, .0250)

ax.legend()

plt.tight_layout()

FigureCanvasNbAgg()

In [70]:


Rsh = 377

kT = 0.05
# Ec = 10
Vc_noise = 50e-6

EJ_list = np.logspace(-2,1.5, 51)

Npts = 251
V_ = np.linspace(-1e-3, 1e-3, Npts)

Isws = []
Isws_conv = []

R0s = []
R0s_conv = []

for EJ in EJ_list:
    Vs, Is, Is_conv = IZ_conv(V_, EJ, Vc_noise , kT, Rsh)
    
#     Isw = (np.max(Is) - np.min(Is))/2
#     Isw_conv = (np.max(Is_conv) - np.min(Is_conv))/2
    
    Imax = np.max(Is)
    Imax_conv = np.max(Is_conv)

    Vmax = Vs[np.argmax(Is)]
    Vmax_conv = Vs[np.argmax(Is_conv)]
    
    R0s.append( Vmax/Imax )
    R0s_conv.append( Vmax_conv/Imax_conv )
    
    
    
    Isws.append(Imax)
    Isws_conv.append(Imax_conv)
    

    


In [14]:
# ft = pd.read_excel("lit_summary.xlsx")
ft = pd.read_excel("exp summary.xlsx")
ft

,ref,"RN, kOhm",Ej,"EJ, K altern.","EC, K","EC, K altern.",Isw,"IAB, nA","R0/JJ, kOhm",sample
0,Shimada 2016,NaN,1.1,NaN,200.0m,NaN,1.2,44.6,110.0m,"SQUID array, frustr=0.375"
1,Goffman 2000,NaN,600.0m,NaN,NaN,0 (C=120pF),18.0,26.0,-,breakjunction shunted by RC
2,Jack 2017,NaN,176.0m,NaN,NaN,0 (C=3nF),50.0m,7.3,400,STM shunted by C
3,Jack PhD 2015 p43,47.9,460.0m,NaN,NaN,NaN,1.5,19.0,13.3,"STM (vanadium, TC=4K)"
4,Jack PhD 2015 p54,2580,9.0m,NaN,NaN,NaN,500.0u,380.0m,32000,"STM (vanadium, TC=4K)"
5,Jack PhD 2015 p56,21,1.0,NaN,NaN,NaN,6.6,44.0,2.5,"STM (vanadium, TC=4K)"
6,Jack PhD 2015 p56,9.6,2.3,NaN,NaN,NaN,25.0,96.0,500.0m,"STM (vanadium, TC=4K)"
7,Jack PhD 2015 p75,100,220.0m,NaN,NaN,NaN,130.0m,9.2,NaN,"STM (vanadium, TC=4K)"
8,Steinbach 2001,7.0,1.1,NaN,7.4,NaN,38.0,44.9,24.0m,"SQUID chain current meter, single JJ with on c..."
9,"Iansiti 1988, 89",140,145.0m,NaN,900.0m,NaN,350.0m,6.1,NaN,"twoJJ or one?, tin, CJ~1fF, C-?"


# Isw vs Ej

In [10]:
# ax = ft.plot( x = 'EJ', y = 'Isw', color ='k', marker = 'D', ls = '')
# ax.set_xscale('log')
# ax.set_yscale('log')

In [71]:
# ax = ft.plot( x = 'EJ', y = 'Isw', color ='k', marker = 'D', ls = '')


fig, ax = plt.subplots(figsize = (8,6))
markers=['d',',','o','v','^','<','>','s','p','P','*','h','H','+','x','X','D','d','|','_','1','2','3','4','8']

unique_lablist = list(set(ft.ref.to_list()))


# for ej, isw, lab, m in zip(ft.Ej, ft.Isw, ft.ref, markers):
    
#     ilab = unique_lablist.index(lab)
#     ax.scatter(ej, isw*1e-9, color = f'C{ilab}',marker = m, label = lab, s = 50)
    
# ax.legend()

ax.set_xscale('log')
ax.set_yscale('log')

# ax.set_xlim(8e-3, 2e1)
# ax.set_ylim(1e-13, 1e-6)

FigureCanvasNbAgg()

In [72]:
ax.set_xlabel('Ej, K')
ax.set_ylabel('sqrt(Isw), A^1/2')

Text(0, 0.5, 'sqrt(Isw), A^1/2')

In [73]:
type(range(0,5))

range

In [74]:
# fig, ax = plt.subplots()


# ax.plot(EJ_list,  EJ_list*kB/hbar*2*e, '--', label = '$A-B$')


ax.plot(EJ_list, Isws, label = 'cIZ')
# ax.plot(EJ_list, Isws_conv, label = 'cIZ + Vc_noise')


ax.set_xscale('log')
ax.set_yscale('log')

leg1 = ax.legend(loc='lower right')

In [75]:
jj79.db_connect()
ids = range(108,158+1)
ZF =  0.004e-3
FF =  0.224e-3
dF = 0.250e-3

jj79.ZF   = ZF
jj79.FF   = ZF+FF


B = np.arange(ZF, ZF + 1.02*dF, 5e-6)
cos = jj79.B_to_cos(B)

Ics = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -650e-6)[0] for idx in ids]
# jj79.Isw_by_id(ids, fullIVC = True, dy = 10e-6, yoff = -650e-6)

# fig, ax = plt.subplots()
ax.plot(4.5*abs(cos), Ics, 'o', label = '79 early IVC Bscan')

# jj85.pbi(739,marker = 'x', axes = ax, label = '85 ')



# jj79.pbi(1728, axes = ax, label = '79 hist')

ax.legend()

# ax.set_xscale('log')
# ax.set_yscale('log')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [76]:
jj70N5.db_connect()

B_list = np.linspace(-.4e-3, .2e-3, 121)
jj70N5.ZF   = -0.093e-3
jj70N5.FF   = 0.130e-3
ids = range(689,809+1)


# B_list = np.linspace(0e-3, .2e-3, 51)
# jj70N5.ZF   = 0e-3
# jj70N5.FF   = 0.220e-3
# ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.410

# l70N5v1, = ax.plot(Ej0*cos_list, np.sqrt(Ics), '.', fillstyle = 'none')
l70N5v1, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

In [77]:
jj70N5.db_connect()

# B_list = np.linspace(-.4e-3, .2e-3, 121)
# jj70N5.ZF   = -0.093e-3
# jj70N5.FF   = 0.130e-3
# ids = range(689,809+1)

B_list = np.linspace(-.10e-3, .5e-3, 121)
# B_list = np.linspace(0e-3, .2e-3, 51)
jj70N5.ZF   = 0e-3
jj70N5.FF   = 0.220e-3
# ids = range(1213, 1263+1)
ids = range(1264, 1384+1)
cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.410

l70N5v2, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')
# l70N5v2, = ax.plot(Ej0*cos_list, np.sqrt(Ics), '.', fillstyle = 'none')

In [78]:
jj70N1.db_connect()

jj70N1.ZF = -2e-6
jj70N1.FF = 205e-6

B_list = np.linspace(-.10e-3, .5e-3, 121)
cos_list = abs (jj70N1.B_to_cos(B_list))

ids = range(14, 134+1)

Ics = [2*extract_Isw_R0_by_id (idx, dy = 30e-6)[0] for idx in ids]

Ej0 = 0.350  
l70N1, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none' )
# l70N1, = ax.plot(Ej0*cos_list, np.sqrt(Ics), '.', fillstyle = 'none' )

In [79]:
jj70N0.db_connect()

jj70N0.ZF = -2e-6
jj70N0.FF = 200e-6


# B_list = np.linspace(-.10e-3, .5e-3, 61)
# cos_list = abs (jj70N0.B_to_cos(B_list))
# ids = range(11, 71+1)

sl = slice(75, 125)
B_list = np.linspace(-1e-3, 1e-3, 201)[sl]
cos_list = abs (jj70N0.B_to_cos(B_list))
ids = range(76, 276+1)[sl]



Ics = [2*extract_Isw_R0_by_id (idx, dy = 20e-6)[0] for idx in ids]

Ej0 = 0.656
l70N0, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')
# l70N0, = ax.plot(Ej0*cos_list, np.sqrt(Ics), '.', fillstyle = 'none')

In [80]:
jj70N0.bpbi(range(76, 276+1))

FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [81]:
# fig, ax = plt.subplots()
    
jj70N7.db_connect()

jj70N7.ZF = 4e-6
jj70N7.FF = 224e-6


B_list = np.linspace(4e-6, 254e-6, 85)
cos_list = abs (jj70N7.B_to_cos(B_list))

ids = range(208, 292+1)



Ics = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -.735e-3)[0] for idx in ids]

Ej0 = 1.18
l70N7, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')
# l70N7, = ax.plot(Ej0*cos_list, np.sqrt(Ics), '.', fillstyle = 'none')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [82]:
jj79N5 = JJmeas(sample = 'D079N5')

jj79N5.db_connect()

cos_list_795 = np.linspace(1, 0, 51 )
Ejs_795 = 4.14*cos_list_795


ids = range(72,122+1)

Ics = [2*extract_Isw_R0_by_id (idx, dy = 30e-6)[0] for idx in ids]
    

Ej0 = 4.14
l79N5, = ax.plot(Ej0*cos_list_795, Ics, '.', fillstyle = 'none')
# l79N5, = ax.plot(Ej0*cos_list_79, np.sqrt(Ics), '.', fillstyle = 'none')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [83]:
jj85 = JJmeas(sample = 'D085N5')

jj85.db_connect()

cos_list_85 = np.arange(0.12, 1.01, 0.01)



ids = np.concatenate([range(592, 623+1),
                range(681, 737+1)])

Is = [2*extract_Isw_R0_by_id (idx, dy = 30e-6)[0] for idx in ids]
    

Ej0 = 5.4
l85N5, = ax.plot(Ej0*cos_list_85, Is, '.', fillstyle = 'none')


In [84]:
l59 = jj59.pbi(940, axes = ax)

In [85]:

cos, Isws = jj57.xy_by_id(184)

Ej0 = 2.5
l57 = ax.plot(Ej0*cos, Isws, 'x')

In [86]:
leg2 = ax.legend([l70N5v1,l70N5v2, l70N1, l70N0, l70N7,l79N5,l85N5, l59, l57 ],
                 ['70N5v1, N=14, Ec = 650mK',
                  '70N5v2, N=14, Ec = 650mK',
                  '70N1, N=15, Ec = ??mK', 
                  '70N0, N=15, Ec = 400mK', 
                  '70N7, N=14, Ec = 200mK', 
                  '79N5, N=1, Ec = 4mK',
                  '85N5, N=1, Ec = 4mK',
                 '59N1, N=12, Ec = ??mK',
                 '57N8, N=20, Ec = ??mK'], 
                 loc='upper left')
# Manually add the first legend back
ax.add_artist(leg1)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Legend does not support <matplotlib.axes._subplots.AxesSubplot object at 0x00000218CB2851D0> instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x00000218CB867CC0>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
  # This is added back by InteractiveShellApp.init_path()


# R0 vs Ej

In [123]:
fig, ax = plt.subplots(figsize = (8,6))



# ax.plot(EJ_list, R0s, label = 'cIZ ???')
# ax.plot(EJ_list, R0s_conv, label = 'cIZ + Vc_noise')


ax.set_xscale('log')
ax.set_yscale('log')

leg1 = ax.legend(loc='upper right')

FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [124]:
jj79.db_connect()
ids = range(108,158+1)
ZF =  0.004e-3
FF =  0.224e-3
dF = 0.250e-3

jj79.ZF   = ZF
jj79.FF   = ZF+FF


B = np.arange(ZF, ZF + 1.02*dF, 5e-6)
cos_list = jj79.B_to_cos(B)

Ej0 = 4.5

R0s, err_R0s = jj79.R0_by_id(ids, fullIVC = True, dy = 50e-6, yoff = -650e-6)
ax.errorbar(Ej0*cos_list, R0s, marker ='.', ls = '', yerr = err_R0s, label = '79N6 single')


ax.legend()


D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [125]:
jj70N5.bpbi(range(689,809+1))

FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [126]:
jj70N5.db_connect()

B_list = np.linspace(-.4e-3, .2e-3, 121)
jj70N5.ZF   = -0.093e-3
jj70N5.FF   = 0.130e-3
ids = range(689,809+1)



cos_list = abs (jj70N5.B_to_cos(B_list))

Ej0 = 0.410

R0s, err_R0s = jj70N5.R0_by_id(ids, fullIVC = False, dy = 50e-6)
ax.errorbar(Ej0*cos_list, R0s/15, marker ='.', ls = '', yerr = err_R0s/15, label = '70N5 N = `15')





<ErrorbarContainer object of 3 artists>

In [127]:
jj70N5.db_connect()

# B_list = np.linspace(-.4e-3, .2e-3, 121)
# jj70N5.ZF   = -0.093e-3
# jj70N5.FF   = 0.130e-3
# ids = range(689,809+1)


B_list = np.linspace(0e-3, .2e-3, 51)
jj70N5.ZF   = 0e-3
jj70N5.FF   = 0.220e-3
ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ej0 = 0.410

R0s, err_R0s = jj70N5.R0_by_id(ids, fullIVC = False, dy = 50e-6)
ax.errorbar(Ej0*cos_list, R0s/15, marker ='.', ls = '', yerr = err_R0s/15, label = '70N5 N = `15')


<ErrorbarContainer object of 3 artists>

In [128]:
jj70N1.db_connect()

jj70N1.ZF = -2e-6
jj70N1.FF = 200e-6

B_list = np.linspace(-.10e-3, .5e-3, 121)
cos_list = abs (jj70N1.B_to_cos(B_list))

ids = range(14, 134+1)


Ej0 = 0.350

R0s, err_R0s = jj70N1.R0_by_id(ids, fullIVC = False, dy = 50e-6)
ax.errorbar(Ej0*cos_list, R0s/15, marker ='.', ls = '', yerr = err_R0s/15, label = '70N1 N = `15')


<ErrorbarContainer object of 3 artists>

In [129]:
jj70N1.bpbi(range(14, 134+1))

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:183: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [130]:
jj70N0.db_connect()

jj70N0.ZF = -2e-6
jj70N0.FF = 200e-6


# B_list = np.linspace(-.10e-3, .5e-3, 61)
# cos_list = abs (jj70N0.B_to_cos(B_list))
# ids = range(11, 71+1)

sl = slice(75, 125)
B_list = np.linspace(-1e-3, 1e-3, 201)[sl]
cos_list = abs (jj70N0.B_to_cos(B_list))
ids = range(76, 276+1)[sl]



Ej0 = 0.656

R0s, err_R0s = jj70N0.R0_by_id(ids, fullIVC = False, dy = 50e-6)
ax.errorbar(Ej0*cos_list, R0s/15, marker ='.', ls = '', yerr = err_R0s/15, label = '70N0 N = `15')


<ErrorbarContainer object of 3 artists>

In [131]:
# fig, ax = plt.subplots()
    
jj70N7.db_connect()

jj70N7.ZF = 4e-6
jj70N7.FF = 224e-6


B_list = np.linspace(4e-6, 254e-6, 85)
cos_list = abs (jj70N7.B_to_cos(B_list))

ids = range(208, 292+1)



# Rs = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -.735e-3)[1] for idx in ids]

Ej0 = 1.18

R0s, err_R0s = jj70N7.R0_by_id(ids, fullIVC = False, dy = 50e-6, yoff = -.735e-3)
ax.errorbar(Ej0*cos_list, R0s/15, marker ='.', ls = '', yerr = err_R0s/15, label = '70N7 N = `15')




D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


<ErrorbarContainer object of 3 artists>

In [132]:
# jj79N5 = JJmeas(sample = 'D079N5')

# jj79N5.db_connect()

# cos_list = np.linspace(1, 0, 51 )



# ids = range(72,122+1)

# # Rs = [1*extract_Isw_R0_by_id (idx, dy = 30e-6)[1] for idx in ids]
    

# Ej0 = 4.14

# R0s, err_R0s = jj79N5.R0_by_id(ids, fullIVC = True, dy = 50e-6)
# ax.errorbar(Ej0*cos_list, R0s/15, marker ='.', ls = '', yerr = err_R0s/15, label = '79N5 N = `15')


In [133]:
jj85 = JJmeas(sample = 'D085N5')

jj85.db_connect()

cos_list = np.arange(0.12, 1.01, 0.01)



ids = np.concatenate([range(592, 623+1),
                range(681, 737+1)])

# Rs = [1*extract_Isw_R0_by_id (idx, dy = 30e-6)[1] for idx in ids]
    

Ej0 = 5.4
R0s, err_R0s = jj85.R0_by_id(ids, fullIVC = False, dy = 50e-6)
ax.errorbar(Ej0*cos_list, R0s, marker ='.', ls = '', yerr = err_R0s, label = '85N5b N = 1')


<ErrorbarContainer object of 3 artists>

In [134]:
l59 = jj59.pbi(941, axes = ax)

In [135]:
ax.legend()

In [136]:
leg2 = ax.legend([l70N5v1,l70N5v2, l70N1, l70N0, l70N7,l79N5,l85N5, l59 ],
                 ['70N5v1, N=14, Ec = 650mK',
                  '70N5v2, N=14, Ec = 650mK',
                  '70N1, N=15, Ec = ??mK', 
                  '70N0, N=15, Ec = 400mK', 
                  '70N7, N=14, Ec = 200mK', 
                  '79N5, N=1, Ec = 4mK',
                  '85N5, N=1, Ec = 4mK',
                 '59N1, N=12, Ec = ?mK'], 
                 loc='lower left')
# Manually add the first legend back
ax.add_artist(leg1)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Legend does not support <matplotlib.axes._subplots.AxesSubplot object at 0x000002A27C851D30> instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
  # Remove the CWD from sys.path while we load stuff.


# IZ for diff T

In [34]:

fig, (ax, ax2) = plt.subplots(1,2, figsize = (10,5))

Rsh = 10e3

for kT in [10e-3, 20e-3, 50e-3, 100e-3, 200e-3, 500e-3]:
    
    # Ec = 10
    Vc_noise = 50e-6

    EJ_list = np.logspace(-2,1.0, 26)

    Npts = 501
    V_ = np.linspace(0, 5e-3, Npts)

    Isws = []
    Isws_conv = []

    R0s = []
    R0s_conv = []

    for EJ in EJ_list:
#         Vs, Is, Is_conv = IZ_conv(V_, EJ, Vc_noise , kT, Rsh)
        
        Is = I_IZ     ( V_, EJ = EJ, R = Rsh, T = kT)
        Vs = V_- Is*Rsh
    #     Isw = (np.max(Is) - np.min(Is))/2
    #     Isw_conv = (np.max(Is_conv) - np.min(Is_conv))/2

        Imax = np.max(Is)
        Imax_conv = np.max(Is_conv)

        Vmax = Vs[np.argmax(Is)]
        Vmax_conv = Vs[np.argmax(Is_conv)]

        R0s.append( Vmax/Imax )
        R0s_conv.append( Vmax_conv/Imax_conv )



        Isws.append(Imax)
        Isws_conv.append(Imax_conv)
        
    ax.plot(EJ_list, Isws, label = 'cIZ T = {:2.0f}mK'.format(kT*1e3))
    ax2.plot(EJ_list, R0s, label = 'cIZ T = {:2.0f}mK'.format(kT*1e3))

    
    #         ax.plot(EJ_list, Isws_conv, label = 'cIZ + Vc_noise T = {:2.0}mK'.format(kT))


ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('EJ, K')
ax.set_ylabel('Isw, A')
ax.legend()

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('EJ, K')
ax2.set_ylabel('R, Ohm')
ax2.legend()

FigureCanvasNbAgg()

In [67]:
ax.set_xlabel('EJ, K')
ax.set_ylabel('Isw, A')

Text(107.49621212121211, 0.5, 'Isw, A')

In [58]:
jj85.pbi(1217, axes = ax)
jj85.pbi(933, axes = ax)
jj85.pbi(739, axes = ax)

In [60]:
jj79.pbi(1727, axes = ax)

In [61]:
ax = jj79.pbi(1726)

# ax.plot()

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

In [63]:
jj79.pbi(1379+4)

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

In [36]:
fig, ax = plt.subplots()

kT = 10e-3
EJ = 0.2

Vs, Is, Is_conv = IZ_conv(V_, EJ, Vc_noise , kT, Rsh)

ax.plot(Vs, Is)

FigureCanvasNbAgg()

In [65]:
jj79.bpbi(range(1625,1656+1))
B = np.linspace(0, 620e-6, 32)

print(B*1e6)

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:165: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


FigureCanvasNbAgg()

No handles with labels found to put in legend.


[  0.  20.  40.  60.  80. 100. 120. 140. 160. 180. 200. 220. 240. 260.
 280. 300. 320. 340. 360. 380. 400. 420. 440. 460. 480. 500. 520. 540.
 560. 580. 600. 620.]


# sqrt(Isw) vs cos

In [70]:
jj85.db_connect()

In [71]:
fig, ax = plt.subplots()

for idx in [1217, 933,739]:
    
    Ej, Isw = xy_by_id(idx)
    
    ax.plot(Ej, np.sqrt(Isw))
    
    

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

In [72]:
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('EJ, K')
ax.set_ylabel('sqrt(Isw), A^1/2')
ax.legend()

No handles with labels found to put in legend.


# comapre D79 and 85

In [8]:
jj85.bpbi(range(603, 623+1))

FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [12]:
jj79.db_connect()

Bs = []
for idx in range(1625,1639+1):
    exp = load_by_run_spec(captured_run_id=idx)
    B = float(exp.exp_name.split('B = ')[1].split(' ')[0])*1e-6
    Bs.append(B)
    print(idx, exp.exp_name)

1625 B = 0 uA_
1626 B = 20 uA_
1627 B = 40 uA_
1628 B = 60 uA_
1629 B = 80 uA_
1630 B = 100 uA_
1631 B = 120 uA_
1632 B = 140 uA_
1633 B = 160 uA_
1634 B = 180 uA_
1635 B = 200 uA_
1636 B = 220 uA_
1637 B = 240 uA_
1638 B = 260 uA_
1639 B = 280 uA_


In [13]:
ids = range(1625,1639+1)

jj79.db_connect()
Ics = [1*extract_Isw_R0_by_id (idx, dy = 20e-6, yoff = 0e-6)[0] for idx in ids]
# jj79.Isw_by_id(ids, fullIVC = True, dy = 10e-6, yoff = -650e-6)

# fig, ax = plt.subplots()


jj79.ZF = 4e-6
jj79.FF = 228e-6

coss = jj79.B_to_cos(np.array(Bs))
ax.plot(abs(coss), Ics, 'o', label = '79 periodicity check')

ax.legend()

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
ax = jj85.pbi(1237, label = 'D085')

jj79.pbi(1629, axes = ax, alpha = .1, label = 'D079')

ax.legend()


ax.set_title('B = 0')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'B = 0')

In [60]:
jj85.B_to_cos(170e-6)

0.3090169943749473

In [15]:
ax = jj85.pbi(1238)
jj85.pbi(722, axes = ax)


jj79.pbi(1629, axes = ax, alpha = .1)


ax.set_title('B = 70')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'B = 70')

In [89]:
ax = jj85.pbi(1239)



jj79.pbi(1632, axes = ax, alpha = .1)


ax.set_title('B = 140')

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'B = 140')

In [16]:
ax = jj85.pbi(479, label = '85N5')



jj79.pbi(1631, axes = ax, alpha = .1,label = '79N6')


ax.set_title('B = 120u, cos ~ 0.65')

ax.legend()

FigureCanvasNbAgg()

In [17]:
ax = jj85.pbi(501, label = '85N5')



jj79.pbi(1634, axes = ax, alpha = .1,label = '79N6')

ax.legend()

ax.set_title('B ~ 170u, cos = .3')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'B ~ 170u, cos = .3')

In [18]:
ax = jj85.pbi(610)



jj79.pbi(142, axes = ax, alpha = .1)


ax.set_title('B = 170')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'B = 170')

In [111]:
jj85.ZF   = -10e-6
jj85.FF   = 215e-6

jj85.B_to_cos(120e-6)

0.6156614753256583

In [107]:
jj79.B_to_cos(180e-6)

0.33027906195516715

In [19]:
cos_list = np.linspace(0.44, 1, 57)

b_list = jj85.cos_to_B(cos_list)

jj85.pbi(range(681, 735+1)[41])

print(b_list[41]*1e6)

Exception: Please indicate value of ZF!

In [66]:
jj79.bpbi(range(108,158+1))

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:165: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [20]:
jj79.db_connect()
ids = range(108,158+1)
ZF =  0.004e-3
FF =  0.224e-3
dF = 0.250e-3

jj79.ZF   = ZF
jj79.FF   = ZF+FF


B = np.arange(ZF, ZF + 1.02*dF, 5e-6)
cos = jj79.B_to_cos(B)

Ics = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -650e-6)[0] for idx in ids]
# jj79.Isw_by_id(ids, fullIVC = True, dy = 10e-6, yoff = -650e-6)

fig, ax = plt.subplots()
ax.plot(abs(cos), Ics, 'o', label = '79 early IVC Bscan')

jj85.pbi(738,marker = 'x', axes = ax, label = '85 ')

print(ids[40])

jj79.pbi(1728, axes = ax, label = '79 hist')

ax.legend()

# ax.set_xscale('log')
# ax.set_yscale('log')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FigureCanvasNbAgg()

148


In [22]:
jj79.bpbi(ids)

FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [67]:
ax.set_xscale('log')
ax.set_yscale('log')

In [21]:
jj79.pbi(1728)

FigureCanvasNbAgg()

In [21]:
jj85.pbi(1415, ls = '-')

FigureCanvasNbAgg()

In [87]:
jj85.ZF = -5e-6
jj85.FF = 250e-6

jj85.db_connect()
B, Isw = xy_by_id(1415)
cos = jj85.B_to_cos(B)

# fig, ax = plt.subplots()
ax.plot( abs(cos), Isw, 'x-', label = '85 periodiciy check')

In [80]:
jj79.cos_to_B(0.3)*1e6

205.50665798966088

In [84]:
jj79.pbi(148)

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

In [86]:
jj79.pbi(1383)
print(load_by_run_spec(captured_run_id=1383).exp_name)

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

6220, cos = 0.30 @T =42.7mK_


In [92]:
exp = load_by_run_spec(captured_run_id=1383)
exp.snapshot

{'station': {'instruments': {'K6220': {'functions': {},
    'submodules': {},
    '__class__': 'qcodes.instrument_drivers.Keithley.Keithley_6220.Keithley_6220',
    'parameters': {'IDN': {'value': {'vendor': 'KEITHLEY INSTRUMENTS INC.',
       'model': '6220',
       'serial': '1249187',
       'firmware': 'A05  /700x'},
      'ts': '2019-12-12 19:53:44',
      'raw_value': {'vendor': 'KEITHLEY INSTRUMENTS INC.',
       'model': '6220',
       'serial': '1249187',
       'firmware': 'A05  /700x'},
      '__class__': 'qcodes.instrument.parameter.Parameter',
      'full_name': 'K6220_IDN',
      'name': 'IDN',
      'instrument': 'qcodes.instrument_drivers.Keithley.Keithley_6220.Keithley_6220',
      'instrument_name': 'K6220',
      'inter_delay': 0,
      'post_delay': 0,
      'vals': '<Anything>',
      'label': 'IDN',
      'unit': ''},
     'timeout': {'value': 10.0,
      'ts': '2019-12-13 21:21:28',
      'raw_value': 10.0,
      '__class__': 'qcodes.instrument.parameter.Paramete

In [102]:
jj79.pbi(1681)
print(load_by_run_spec(captured_run_id=1681).exp_name)

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

keith, cos = 0.20 @T =41.8mK_


In [8]:
for idx in range(1000, 1300):
    print(idx , load_by_run_spec(captured_run_id=idx).exp_name)

1000 Bscan IVC @T =439.3mK, B = 169.0 µA_
1001 Bscan IVC @T =443.6mK, B = 174.0 µA_
1002 Bscan IVC @T =445.4mK, B = 179.0 µA_
1003 Bscan IVC @T =450.2mK, B = 184.0 µA_
1004 Bscan IVC @T =438.8mK, B = 189.0 µA_
1005 Bscan IVC @T =449.9mK, B = 194.0 µA_
1006 Bscan IVC @T =443.4mK, B = 199.0 µA_
1007 Bscan IVC @T =459.2mK, B = 204.0 µA_
1008 Bscan IVC @T =457.0mK, B = 209.0 µA_
1009 Bscan IVC @T =462.8mK, B = 214.0 µA_
1010 Bscan IVC @T =441.9mK, B = 219.0 µA_
1011 Bscan IVC @T =438.9mK, B = 224.0 µA_
1012 Bscan IVC @T =458.1mK, B = 229.0 µA_
1013 Bscan IVC @T =445.1mK, B = 234.0 µA_
1014 Bscan IVC @T =452.4mK, B = 239.0 µA_
1015 Bscan IVC @T =453.2mK, B = 244.0 µA_
1016 Bscan IVC @T =444.3mK, B = 249.0 µA_
1017 Bscan IVC @T =453.1mK, B = 254.0 µA_
1018 Bscan IVC @T =543.9mK, B = 4.0 µA_
1019 Bscan IVC @T =551.3mK, B = 9.0 µA_
1020 Bscan IVC @T =552.7mK, B = 14.0 µA_
1021 Bscan IVC @T =560.2mK, B = 19.0 µA_
1022 Bscan IVC @T =559.2mK, B = 24.0 µA_
1023 Bscan IVC @T =547.3mK, B = 29.0 µA_


In [91]:
for idx in range(1600, 1700):
    print(idx , load_by_run_spec(captured_run_id=idx).exp_name)

1600 cos = 0.15_
1601 cos = 0.15_
1602 cos = 0.15_
1603 cos = 0.15_
1604 cos = 0.12_
1605 cos = 0.10_
1606 cos = 0.07_
1607 cos = 0.05_
1608 cos = 0.03_
1609 cos = 0.00_
1610 cos = 0.00_
1611 cos = 0.00_
1612 cos = 0.00_
1613 cos = 0.00_
1614 cos = 0.00_
1615 cos = 0.00_
1616 cos = 0.00_
1617 cos = 0.00_
1618 cos = 0.00_
1619 cos = 0.00_
1620 cos = 0.00_
1621 cos = 0.00_
1622 cos = 0.00_
1623 cos = 0.00_
1624 cos = 0.00_
1625 B = 0 uA_
1626 B = 20 uA_
1627 B = 40 uA_
1628 B = 60 uA_
1629 B = 80 uA_
1630 B = 100 uA_
1631 B = 120 uA_
1632 B = 140 uA_
1633 B = 160 uA_
1634 B = 180 uA_
1635 B = 200 uA_
1636 B = 220 uA_
1637 B = 240 uA_
1638 B = 260 uA_
1639 B = 280 uA_
1640 B = 300 uA_
1641 B = 320 uA_
1642 B = 340 uA_
1643 B = 360 uA_
1644 B = 380 uA_
1645 B = 400 uA_
1646 B = 420 uA_
1647 B = 440 uA_
1648 B = 460 uA_
1649 B = 480 uA_
1650 B = 500 uA_
1651 B = 520 uA_
1652 B = 540 uA_
1653 B = 560 uA_
1654 B = 580 uA_
1655 B = 600 uA_
1656 B = 620 uA_
1657 B = 208 uA_
1658 B = 208 uA_
165

## IVC hist B = 174

In [62]:
jj79.B_to_cos(174e-6)

0.48175367410171516

In [82]:
ax = jj79.bpbi([142, 1251, 1457], labels = ['B = 174', 'B = 174', 'B = 174'])
ax.legend()
print(idx , load_by_run_spec(captured_run_id=142).exp_name)

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:165: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


FigureCanvasNbAgg()

1499 Bscan IVC @T =50.5mK, B = 174.0 µA_


In [49]:
ax = jj79.pbi(1449, label = 'B = 172')
ax.legend()

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

## IVC hist B = 70

In [57]:
ax = jj79.pbi(121, label = 'B = 70')
ax.legend()

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

In [56]:
ax = jj79.pbi(1444, label = 'B = 70')
ax.legend()

D:\anaconda\lib\site-packages\qcodes\dataset\plotting.py:199: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, **subplots_kwargs)


FigureCanvasNbAgg()

## 79N6 periodicy

In [10]:
ids = range(1234, 1284+1)

jj79.bpbi(ids)

FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [16]:
jj79.FF

0.00022799999999999999

In [19]:



B = np.arange(4e-6, 514e-6, 10e-6)
cos = jj79.B_to_cos(B)

Ics = [1*extract_Isw_R0_by_id (idx, dy = 30e-6, yoff = -576e-6)[0] for idx in ids]
# jj79.Isw_by_id(ids, fullIVC = True, dy = 10e-6, yoff = -650e-6)

fig, ax = plt.subplots()
ax.plot(abs(cos), Ics, 'o')

# jj85.pbi(738, axes = ax)

# print(ids[40])

# jj79.pbi(1728, axes = ax)

# ax.set_xscale('log')
# ax.set_yscale('log')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


len is zero


FigureCanvasNbAgg()

In [ ]:
jj.pbi(1726)

# sandbox

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:165: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


FigureCanvasNbAgg()

No handles with labels found to put in legend.


In [251]:
jj70N7.ZF = -.002e-3
jj70N7.FF = .220e-3

jj70N7.cos_to_B(0.0)

0.00021999999999999998

In [226]:
fig, ax = plt.subplots()
    
jj70N7.db_connect()

jj70N7.ZF = 4e-6
jj70N7.FF = 224e-6


B_list = np.linspace(4e-6, 254e-6, 85)
cos_list = abs (jj70N7.B_to_cos(B_list))

ids = range(208, 292+1)



Ics = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -.735e-3)[0] for idx in ids]

Ej0 = 1.18
ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


len is zero
len is zero
len is zero
len is zero
len is zero
len is zero
len is zero
len is zero


In [215]:
jj = jj70N0
jj70N0.db_connect()

jj.ZF = -0e-6
jj.FF = 210e-6

sl = slice(75, 125)

B_list = np.linspace(-1e-3, 1e-3, 201)[sl]
cos_list = abs (jj.B_to_cos(B_list))

ids = range(76, 276+1)[sl]

Ics = [2e12*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]


fig, ax = plt.subplots()
ax.plot(cos_list, Ics, 'o-')
# ax.plot(B_list, Ics, 'o-')

# ax.set_xscale('log')
# ax.set_xlim(0.1, 1)
# ax.set_yscale('log')

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )
D:\anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


FigureCanvasNbAgg()

In [242]:
fig, ax = plt.subplots()

jj70N5.db_connect()

B_list = np.linspace(-.4e-3, .2e-3, 121)
jj70N5.ZF   = -0.093e-3
jj70N5.FF   = 0.130e-3
ids = range(689,809+1)


# B_list = np.linspace(0e-3, .2e-3, 51)
# jj70N5.ZF   = 0e-3
# jj70N5.FF   = 0.220e-3
# ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.410

ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none', label = '70N5 v1')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


In [243]:
jj70N5.db_connect()

# B_list = np.linspace(-.4e-3, .2e-3, 121)
# jj70N5.ZF   = -0.093e-3
# jj70N5.FF   = 0.130e-3
# ids = range(689,809+1)


B_list = np.linspace(0e-3, .2e-3, 51)
jj70N5.ZF   = 0e-3
jj70N5.FF   = 0.220e-3
ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.34

ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none', label = '70N5 v2')

ax.legend()

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlim(0.05, .5)

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


(0.05, 0.5)